In [1]:
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import pandas as pd
import numpy as np

In [2]:
#web scraping the page links
req = Request("https://www.formula1.com/en/results.html/2021/races.html")
html_page = urlopen(req)


soup = BeautifulSoup(html_page, "lxml")

links = []
for link in soup.findAll('a'):
    links.append(link.get('href'))



In [3]:
# print(links)

filter = "/en/results.html/2021/races/1" 
cleaned_list = []
for i in links:
    if i != None:
        if filter in i and i not in cleaned_list:
            cleaned_list.append(i)

links_2021 = ["https://www.formula1.com/"+i for i in cleaned_list]



In [4]:
#building data
data = []
for i in links_2021:
    data.append(pd.read_html(i))

races = []
race_counter = 1
for race in data:
    cleaned_race = race[0][['Car','PTS']]
    cleaned_race.set_index('Car',inplace=True) #set indexes the name of driver
    cleaned_race = cleaned_race.groupby(by=["Car"]).sum()
    cleaned_race.rename({'PTS' : str(race_counter)}, axis = 1, inplace = True)
    race_counter += 1
    races.append(cleaned_race)

drivers = set(races[1].index)
for i in races:
    for name in list(i.index):
        if name not in drivers:
            drivers.add(name)

points = pd.DataFrame(drivers)
points.set_index(points[0],inplace=True)
points = points.join(races[0])
points = points.drop(labels=[0],axis=1)
for i in range(1,len(races)):
    points = points.join(races[i])

In [5]:
points

,1,2,3,4,5,6,7,8,9,10,...,13,14,15,16,17,18,19,20,21,22
0,,,,,,,,,,,,,,,,,,,,,
Red Bull Racing Honda,28,25,30,29,37,25,41,37,34,0,...,29,10,20,33,40,40,31,31,18,26
Alpine Renault,0,3,10,2,2,8,4,2,1,8,...,10,5,8,1,0,2,6,25,12,6
AlphaTauri Honda,2,6,1,1,8,21,6,1,2,1,...,12,0,0,8,2,12,6,0,8,22
Ferrari,12,22,8,18,18,16,0,14,14,26,...,16,20,15,16,18,18,18,10,10,16
McLaren Mercedes,18,23,12,12,15,12,18,10,21,22,...,1,44,19,6,14,1,1,2,11,6
Williams Mercedes,0,0,0,0,0,0,0,0,0,0,...,0,2,1,0,0,0,0,0,0,0
Aston Martin Mercedes,1,4,0,0,14,18,3,4,0,4,...,0,6,0,2,1,6,0,9,0,0
Mercedes,41,19,41,40,7,0,30,34,30,40,...,34,15,35,36,27,18,40,25,41,26
Haas Ferrari,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
def init_dict():
    drivers = {i : 0 for i in list(points.index)}
    for key in drivers.keys():
        drivers[key] = {i : 0 for i in list(points.index)}
    return(drivers)
overall_drivers = init_dict()


for agonas in np.asarray(points.T):
    drivers = init_dict()
    cnt = 0
    for driver in drivers:
        sum_loss = 0
        driver_score = agonas[cnt]
        cnt += 1
        cnt2 = 0
        for score in agonas:
            if driver_score <= score:
                drivers[driver][list(drivers.keys())[cnt2]] += 1
                sum_loss += 1
            cnt2 += 1
        if sum_loss != 0:
            for key in overall_drivers[driver]:
                overall_drivers[driver][key] += drivers[driver][key]/sum_loss

In [7]:
pd.DataFrame(overall_drivers).T

,Red Bull Racing Honda,Alpine Renault,AlphaTauri Honda,Ferrari,McLaren Mercedes,Williams Mercedes,Aston Martin Mercedes,Mercedes,Haas Ferrari,Alfa Romeo Racing Ferrari
Red Bull Racing Honda,15.016667,0.266667,0.266667,0.516667,0.350000,0.266667,0.100000,5.016667,0.1,0.100000
Alpine Renault,3.664286,4.914286,2.014286,2.654762,2.704762,0.342857,1.414286,3.747619,0.2,0.342857
AlphaTauri Honda,4.142857,1.809524,4.809524,3.059524,2.526190,0.466667,0.976190,3.609524,0.3,0.300000
Ferrari,5.291667,1.125000,0.975000,6.541667,2.208333,0.225000,0.475000,4.958333,0.1,0.100000
McLaren Mercedes,5.041667,1.141667,1.125000,2.708333,6.375000,0.350000,0.591667,4.341667,0.1,0.225000
Williams Mercedes,2.585714,2.285714,2.000000,2.285714,2.085714,2.785714,1.942857,2.285714,1.8,1.942857
Aston Martin Mercedes,3.316667,1.466667,2.266667,2.816667,2.783333,1.100000,3.516667,2.933333,0.9,0.900000
Mercedes,4.016667,0.766667,0.600000,1.183333,0.850000,0.433333,0.266667,13.683333,0.1,0.100000
Haas Ferrari,2.200000,2.200000,2.200000,2.200000,2.200000,2.200000,2.200000,2.200000,2.2,2.200000
Alfa Romeo Racing Ferrari,2.486905,2.320238,2.320238,2.486905,2.177381,1.742857,2.034524,2.344048,1.6,2.486905


In [8]:
p=np.asarray(pd.DataFrame(overall_drivers).T)
s=np.ones((10,10))
Q=0.85*p+(0.15)*(1/10)*s



In [9]:
#poewred method
z0=(1/10)*np.ones((1,10))

In [10]:
from numpy.linalg import matrix_power
q=z0@matrix_power(Q,100)

In [11]:
vec = pd.DataFrame(q)

In [12]:
vecT=vec.T

In [13]:
vecT.insert(0,"names", points.index, True)

In [14]:
vecT.sort_values(0)

,names,0
8,Haas Ferrari,3.373283e+125
9,Alfa Romeo Racing Ferrari,3.782002e+125
5,Williams Mercedes,7.076224e+125
6,Aston Martin Mercedes,7.361465e+125
2,AlphaTauri Honda,1.355397e+126
1,Alpine Renault,1.452680e+126
4,McLaren Mercedes,2.098821e+126
3,Ferrari,2.567715e+126
7,Mercedes,1.186450e+127
0,Red Bull Racing Honda,1.247471e+127
